In [ ]:
words = ['apple', 'banana', 'cherry', 'date', 'elderberry']

words_map = {word:idx for idx, word in enumerate(words)}

def get_one_hot(word):
    if word not in words_map:
        raise ValueError(f"Word '{word}' not found in the list.")
    
    one_hot = [0] * len(words)
    one_hot[words_map[word]] = 1
    return one_hot

for word in words:
    print(f'{word}: {get_one_hot(word)}')


In [ ]:
from openai import OpenAI
import json
client = OpenAI(api_key="sk-acUzWFHVvFTIaINs4fF7Bc2b30F24b45AdC97dAa6aAe8cEa", base_url="https://dd-ai-api.eastmoney.com/v1")

tools = [{"type":"function",
          "function":{
               "name": "get_weather",
               "description": "Get current temperature for a given location.",
               "parameters":{
                   "type":"object",
                   "properties": {
                       "location": {
                           "type":"string",
                            "description": "City and country e.g. Bogotá, Colombia"
                            }
                    },
                    "required": ["location"],
                    "additionalProperties": False
                
                },
               "strict": True
           }
       }]
messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

completion = client.chat.completions.create(model="gpt-4o",messages=messages,tools=tools)

print(completion.choices[0].message.content)

In [ ]:
from langchain.chains import LLMMathChain, LLMChain
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.prompts import PromptTemplate

from langchain import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "sk-acUzWFHVvFTIaINs4fF7Bc2b30F24b45AdC97dAa6aAe8cEa"
os.environ["OPENAI_API_BASE"] = "https://dd-ai-api.eastmoney.com/v1"

llm = OpenAI(model_name="Private-Qwen3-235B-A22B", temperature=0)

llm_math = LLMMathChain.from_llm(llm=llm)

# 初始化math_tool
math_tool = Tool(
    name="Calculator",
    func=llm_math.run,
    description="Useful for when you need to answer questions about math."
)

prompt = PromptTemplate(
    input_variables=["question"],
    template = """
    你是一个数学助手。请回答以下问题：
    {question}
    """)

search_chain = LLMChain(llm=llm, prompt=prompt)

search_tool = Tool(
    name = '商品信息查询工具',
    func=search_chain.run,
    description="查询商品的价格、库存等信息"
)

tools = [math_tool, search_tool]


zero_shot_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

zero_shot_agent.invoke({"input":"一个商品原价100元，打8折出售，现在买这个商品需要多少钱？"})

In [ ]:
from langchain.chains import LLMMathChain, LLMChain
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_core.prompts import PromptTemplate
from langchain import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "sk-acUzWFHVvFTIaINs4fF7Bc2b30F24b45AdC97dAa6aAe8cEa"
# os.environ["OPENAI_API_KEY"] = "sk-BmXkizlJvANUeObY1365B92287Be4c66A1888448D9D1B689"
os.environ["OPENAI_API_BASE"] = "https://dd-ai-api.eastmoney.com/v1"

# 初始化LLM - 新版本写法
llm = OpenAI(model_name="Private-Qwen3-235B-A22B", temperature=0)

# 数学工具链
llm_math = LLMMathChain.from_llm(llm=llm)

# 初始化math_tool
math_tool = Tool(
    name="Calculator",
    func=llm_math.run,
    description="Useful for when you need to answer questions about math."
)

# 提示模板
prompt = PromptTemplate(
    input_variables=["input"],
    template="""你需要根据以下信息回复用户关于商品价格的查询，如果你无法从以下信息中解答用户问题，请说我不知道。
    商品信息列表如下：
        可口可乐5元1瓶
        百事可乐3元1瓶
    这是用户输入的问题：
    {input}"""
)

# 搜索链
search_chain = LLMChain(llm=llm, prompt=prompt)

search_tool = Tool(
    name='商品信息查询工具',
    func=search_chain.run,
    description="用于专门查询商品的价格信息"
)

tools = [math_tool, search_tool]

# 初始化agent - 新版本写法
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 执行agent
result = agent.invoke({"input": "可口可乐的价格的三次方是多少？"})
print(result["output"])

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

from dotenv import load_dotenv

load_dotenv()

def create_agent(llm, system_message: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "你是一个协作的AI助手，负责执行分配的任务并提供有用的输出。"
                "在完成任务后，请在回复最后加上'FINAL ANSWER:'。以下是你的任务:{system_message}"
            ),
            MessagesPlaceholder(variable_name = 'messages')
        ]
    )
    prompt = prompt.partial(system_message = system_message)
    return prompt | llm

llm = ChatOpenAI(model='gpt-4o', temperature = 0)
agent_1 = create_agent(llm, "生成10个随机数字")
agent_2 = create_agent(llm, "请将提供的数字每个都乘以10")


from langgraph.graph import StateGraph, END
from typing import Dict, Any, List, TypedDict
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: List[BaseMessage]
    sender: str

graph_builder = StateGraph(AgentState)

# 添加节点处理函数
def call_agent_1(state):
    response = agent_1.invoke(state)
    # 更新状态
    return {
        "messages": state["messages"] + [response],
        "sender": "agent_1"
    }

def call_agent_2(state):
    response = agent_2.invoke(state)
    # 更新状态
    return {
        "messages": state["messages"] + [response],
        "sender": "agent_2"
    }

# 添加节点
graph_builder.add_node("agent_1", call_agent_1)
graph_builder.add_node("agent_2", call_agent_2)

# 简化的路由函数
def simple_router(state):
    # 访问字典的键而不是属性
    messages = state["messages"]
    
    if not messages:
        return "agent_1"
    
    last_message = messages[-1]
    current_sender = state["sender"]

    # 如果包含FINAL ANSWER，则转到下一个agent或结束
    if "FINAL ANSWER:" in last_message.content:
        if current_sender == "agent_1":
            return "agent_2"
        else:
            return END
    else:
        # 如果没有FINAL ANSWER，继续当前agent（但这通常不应该发生）
        if current_sender == "agent_1":
            return "agent_1"
        else:
            return "agent_2"

# 设置边
graph_builder.add_conditional_edges("agent_1", simple_router)
graph_builder.add_conditional_edges("agent_2", simple_router)

graph_builder.set_entry_point("agent_1")

graph = graph_builder.compile()

# 运行
initial_state = {
    "messages": [HumanMessage(content="请生成10个随机数字，并将每个数乘以10")],
    "sender": "agent_1"
}

events = graph.stream(initial_state)
for event in events:
    print(event)
    print("------")

{'agent_1': {'messages': [HumanMessage(content='请生成10个随机数字，并将每个数乘以10', additional_kwargs={}, response_metadata={}), AIMessage(content='以下是生成的10个随机数字，并将每个数字乘以10后的结果：\n\n1. 原始数字：7，乘以10后：70  \n2. 原始数字：3，乘以10后：30  \n3. 原始数字：9，乘以10后：90  \n4. 原始数字：5，乘以10后：50  \n5. 原始数字：2，乘以10后：20  \n6. 原始数字：8，乘以10后：80  \n7. 原始数字：4，乘以10后：40  \n8. 原始数字：6，乘以10后：60  \n9. 原始数字：1，乘以10后：10  \n10. 原始数字：10，乘以10后：100  \n\nFINAL ANSWER: [70, 30, 90, 50, 20, 80, 40, 60, 10, 100]', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 204, 'prompt_tokens': 72, 'total_tokens': 276, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-C3ZnLi4SQUsEsRARsYRUdyrWIe8Fd', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--065eac

In [4]:
from langgraph.graph import StateGraph, END, START
from langgraph.graph import MessagesState

graph_builder = StateGraph(MessagesState)

# 添加节点
graph_builder.add_node("agent_1", agent_1)
graph_builder.add_node("agent_2", agent_2)

# 简化的路由函数
def simple_router(state):
    messages = state["messages"]
    if not messages:
        return "agent_1"
    
    last_message = messages[-1]
    
    # 如果是第一次运行或者来自agent_1，且包含Final ANSWER
    if "Final ANSWER" in last_message.content:
        # 检查消息数量来判断当前应该转到哪里
        if len(messages) == 2:  # 第一个agent完成
            return "agent_2"
        else:  # 第二个agent完成
            return END
    
    return "continue"  # 其他情况结束

# 设置边
graph_builder.add_edge(START, "agent_1")
graph_builder.add_conditional_edges("agent_1", simple_router)
graph_builder.add_conditional_edges("agent_2", simple_router)

graph = graph_builder.compile()

# 运行
initial_state = {
    "messages": [HumanMessage(content="请生成10个随机数字,并将每个数字乘以10")]
}

events = graph.stream(initial_state, {"recursion_limit": 50})

for event in events:
    print(event)
    print("------")

Task agent_1 with path ('__pregel_pull', 'agent_1') wrote to unknown channel branch:to:continue, ignoring it.


{'agent_1': None}
------
